In [8]:
#import necessary libraries
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
from PIL import Image

In [9]:
# data cleaning
batch_size = 32
image_height = 128
image_width = 128

directory = 'pokemon_dataset/pokemon_dataset/'

# change all jpg's to pngs
for filename in os.listdir(directory):
    if filename.endswith(".jpg"):
        prefix = filename.split(".jpg")[0]
        os.rename(os.path.join(directory, filename), os.path.join(directory, prefix+".png"))
# remove all gifs from data
    elif filename.endswith(".gif"):
        os.remove(os.path.join(directory, filename))
    print(filename)


Abomasnow_0_0.png
Abomasnow_0_1.png
Abomasnow_0_2.png
Abomasnow_0_3.png
Abomasnow_1_0.png
Abomasnow_1_1.png
Abomasnow_1_2.png
Abomasnow_1_3.png
Abomasnow_2_0.png
Abomasnow_2_1.png
Abomasnow_2_2.png
Abomasnow_2_3.png
Abomasnow_3_0.png
Abomasnow_3_1.png
Abra_0_0.png
Abra_0_1.png
Abra_0_2.png
Abra_0_3.png
Abra_1_0.png
Abra_1_1.png
Abra_1_2.png
Abra_1_3.png
Abra_2_0.png
Abra_2_1.png
Abra_2_2.png
Abra_2_3.png
Abra_3_0.png
Abra_3_1.png
Absol_0_0.png
Absol_0_1.png
Absol_0_2.png
Absol_0_3.png
Absol_1_0.png
Absol_1_1.png
Absol_1_2.png
Absol_1_3.png
Absol_2_0.png
Absol_2_1.png
Absol_2_2.png
Absol_2_3.png
Absol_3_0.png
Absol_3_1.png
Accelgor_0_0.png
Accelgor_0_1.png
Accelgor_0_2.png
Accelgor_0_3.png
Accelgor_1_0.png
Accelgor_1_1.png
Accelgor_2_0.png
Accelgor_2_1.png
Accelgor_3_0.png
Accelgor_3_1.png
Aegislash_0_0.png
Aegislash_0_1.png
Aegislash_1_0.png
Aegislash_1_1.png
Aegislash_2_0.png
Aegislash_2_1.png
Aegislash_3_0.png
Aegislash_3_1.png
Aerodactyl_0_0.png
Aerodactyl_0_1.png
Aerodactyl_0_2.png

In [10]:
# Get all unique labels
pokemon_names = [name.split('_')[0] for name in os.listdir(directory)]
pokemon_names = sorted(set(pokemon_names))  # remove duplicates
name_to_int = {name: i for i, name in enumerate(pokemon_names)}
print(name_to_int)

{'Abomasnow': 0, 'Abra': 1, 'Absol': 2, 'Accelgor': 3, 'Aegislash': 4, 'Aerodactyl': 5, 'Aggron': 6, 'Aipom': 7, 'Alakazam': 8, 'Alcremie': 9, 'Alomomola': 10, 'Altaria': 11, 'Amaura': 12, 'Ambipom': 13, 'Amoonguss': 14, 'Ampharos': 15, 'Annihilape': 16, 'Anorith': 17, 'Appletun': 18, 'Applin': 19, 'Araquanid': 20, 'Arbok': 21, 'Arboliva': 22, 'Arcanine': 23, 'Arceus': 24, 'Archaludon': 25, 'Archen': 26, 'Archeops': 27, 'Arctibax': 28, 'Ariados': 29, 'Armaldo': 30, 'Armarouge': 31, 'Aromatisse': 32, 'Aron': 33, 'Arrokuda': 34, 'Articuno': 35, 'Audino': 36, 'Aurorus': 37, 'Avalugg': 38, 'Axew': 39, 'Azelf': 40, 'Azumarill': 41, 'Azurill': 42, 'Bagon': 43, 'Baltoy': 44, 'Banette': 45, 'Barbaracle': 46, 'Barboach': 47, 'Barraskewda': 48, 'Basculegion': 49, 'Basculin': 50, 'Bastiodon': 51, 'Baxcalibur': 52, 'Bayleef': 53, 'Beartic': 54, 'Beautifly': 55, 'Beedrill': 56, 'Beheeyem': 57, 'Beldum': 58, 'Bellibolt': 59, 'Bellossom': 60, 'Bellsprout': 61, 'Bergmite': 62, 'Bewear': 63, 'Bibarel':

In [11]:
# Processing the path to get the label
def process_path(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3) 
    image = tf.image.resize(image, [image_height, image_width])
    label = tf.strings.split(file_path, '\\')[-1]  # get the last part after splitting by '\\'
    label = tf.strings.split(label, '_')[0]
    
    # Define a Python function to convert tensor to int
    def label_to_int(label_tensor):
        label_str = label_tensor.numpy().decode('utf-8')
        return name_to_int[label_str]

    # Apply the Python function using tf.py_function
    label = tf.py_function(label_to_int, inp=[label], Tout=tf.int64)
    label.set_shape([])
        
    return image, label

In [16]:
# Creating Model
model = keras.Sequential([
    layers.Input((image_height, image_width, 3)),
    layers.Conv2D(16, 3, padding = 'same', activation ='relu'),
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(971)
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ["accuracy"]
)

ds_train = tf.data.Dataset.list_files(str(pathlib.Path(directory+'*.png')))
ds_train = ds_train.map(process_path).batch(batch_size)

model.summary()

model.fit(ds_train, epochs=10, verbose = 3)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 conv2d_13 (Conv2D)          (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 131072)            0         
                                                                 
 dense_6 (Dense)             (None, 971)               127271883 
                                                                 
Total params: 127,276,971
Trainable params: 127,276,971
Non-trainable params: 0
________________________________________

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "D:\Anaconda3\envs\pokeVenv\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\Anaconda3\envs\pokeVenv\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "D:\Anaconda3\envs\pokeVenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\Anaconda3\envs\pokeVenv\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "D:\Anaconda3\envs\pokeVenv\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "D:\Anaconda3\envs\pokeVenv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "D:\Anaconda